# Duqduq demo: large scale validation

This notebook shows how to use [duqtools](https://duqtools.readthedocs.org) to large scale validation.

It will go over the steps required to do uncertainty quantification from a sequence of data sets.

Where `duqtools` does UQ for a single data set, `duqduq` loops over multiple datasets to do UQ in sequence.

We define 2 directories:

- **duqduq directory**, this is where the duqtools and UQ config resides. This is also the directory we work in with duqduq.
- **run directory**, this is a directory where slurm has access and where all the simulation files and data are stored.

In [2]:
from pathlib import Path
duqtools_dir = Path('/afs/eufus.eu/user/g/g2ssmee/duqduq_demo')
run_dir = Path('/afs/eufus.eu/user/g/g2ssmee/jetto_runs/duqduq_demo')

import os
os.chdir(duqtools_dir)

## `duqduq help`

The main interface for duqduq is via the CLI. You can run `duqduq --help` to give a list of available subcommands.

You will notice that the subcommands here mimic what is available in `duqtools`.

In [3]:
!duqduq --help

Usage: duqduq [OPTIONS] COMMAND [ARGS]...

  For more information, check out the documentation:

  https://duqtools.readthedocs.io/large_scale_validation

Options:
  --help  Show this message and exit.

Commands:
  create  Create data sets for large scale validation.
  merge   Merge data sets with error propagation.
  setup   Set up large scale validation.
  status  Check status large scale validation runs.
  submit  Submit large scale validation runs.


## `duqduq setup`

The starting point for `duqduq` is 2 files:

- `duqtools.template.yaml`, this is the template config that `duqduq setup` will use to generate the `duqtools.yaml`
- `data.csv`, each entry in this csv file corresponds to an IMAS data set

Below is an example `data.csv` file. This is how you tell `duqduq` which data to do UQ for.

In [8]:
%cat data.csv

,user,db,shot,run
data_01,g2aho,aug,36982,0002
data_02,g2aho,jet,75225,0002
data_03,g2aho,jet,90350,0002
data_04,g2aho,jet,92432,0002
data_05,g2aho,jet,94875,0001
data_06,g2aho,tcv,64958,0002
data_07,g2aho,west,54568,0001
data_08,g2aho,west,54728,0001
data_09,g2aho,west,55181,0001
data_10,g2aho,west,55525,0001


Below is an example `duqtools.template.yaml`.

The index of each entry in `data.csv` file will be used as the run name (`run.name`).

The details for each entry in `data.csv` will be written to the `template_data` section.

Machine/dataset specific parameters, such as major radius or the start time are grabbed from the IDS. 

For more information, see the [documentation](https://duqtools.readthedocs.io/en/latest/large_scale_validation/) for large scale validation.

In [9]:
%cat duqtools.template.yaml

tag: {{ run.name }}
create:
  runs_dir: /afs/eufus.eu/user/g/g2ssmee/jetto_runs/duqduq_long/{{ run.name }}
  template: /afs/eufus.eu/user/g/g2ssmee/jetto_runs/interpretive_esco02
  template_data:
    user: {{ handle.user }}
    db: {{ handle.db }}
    shot: {{ handle.shot }}
    run: {{ handle.run }}
  sampler:
    method: latin-hypercube
    n_samples: 9
  dimensions:
    - variable: zeff
      operator: multiply
      values: [0.8, 1.0, 1.2]
    - variable: t_e
      operator: multiply
      values: [0.8, 1.0, 1.2]
    - variable: major_radius
      operator: copyto
      {# Convert units from IDS (m) to Jetto.jset (cm) -#}
      values: [ {{ (variables.major_radius * 100) | round(4) }} ]
    - variable: b_field
      operator: copyto
      values: [ {{ variables.b_field | round(4) }} ]
    - variable: t_start
      operator: copyto
      values: [ {{ variables.t_start | round(4) }} ]
    - variable: t_end
      operator: copyto
      values: [ {{ (variables.t_start + 1.0) | round(4)

Running `duqduq setup` will generate a new directory for each dataset in `data.csv`. Each directory is in itself a valid **duqtools directory**.

In [11]:
!duqduq setup --yes


Operations in the Queue:
- Setup run : data_01
- Setup run : data_02
- Setup run : data_03
- Setup run : data_04
- Setup run : data_05
- Setup run : data_06
- Setup run : data_07
- Setup run : data_08
- Setup run : data_09
- Setup run : data_10
Applying Operations
10

  0%|                                                    | 0/10 [00:00<?, ?it/s]
Setup run : data_10: 
Progress: 100%|████████████████████████████████| 10/10 [00:00<00:00, 393.28it/s]


This is what the directory looks like after setup.

In [13]:
!tree .

.
├── data_01
│   └── duqtools.yaml
├── data_02
│   └── duqtools.yaml
├── data_03
│   └── duqtools.yaml
├── data_04
│   └── duqtools.yaml
├── data_05
│   └── duqtools.yaml
├── data_06
│   └── duqtools.yaml
├── data_07
│   └── duqtools.yaml
├── data_08
│   └── duqtools.yaml
├── data_09
│   └── duqtools.yaml
├── data_10
│   └── duqtools.yaml
├── data.csv
├── duqtools.log
└── duqtools.template.yaml

10 directories, 13 files


It creates a duqtools config in each of the subdirectories. At this stage you could modify each of the `duqtools.yaml` if you wish. The config is no different than for a single UQ run. This means you could do`cd data_01` and treat it as a single UQ run.

In [14]:
%cat data_01/duqtools.yaml

tag: data_01
create:
  runs_dir: /afs/eufus.eu/user/g/g2ssmee/jetto_runs/duqduq_long/data_01
  template: /afs/eufus.eu/user/g/g2ssmee/jetto_runs/interpretive_esco02
  template_data:
    user: g2aho
    db: aug
    shot: 36982
    run: 2
  sampler:
    method: latin-hypercube
    n_samples: 9
  dimensions:
    - variable: zeff
      operator: multiply
      values: [0.8, 1.0, 1.2]
    - variable: t_e
      operator: multiply
      values: [0.8, 1.0, 1.2]
    - variable: major_radius
      operator: copyto
      values: [ 165.0 ]
    - variable: b_field
      operator: copyto
      values: [ -2.5725 ]
    - variable: t_start
      operator: copyto
      values: [ 2.875 ]
    - variable: t_end
      operator: copyto
      values: [ 3.875 ]
system: jetto-v220922

## Create runs using `duqduq create`

This is the equivalent of `duqtools create`, but for a large number of runs.

It will take each of the duqtools configs generated and set up the jetto runs and imas data according to the specification.

Since this will take a long time, we will use the `--dry_run` option.

In [15]:
!duqduq create --yes --dry_run

Usage: duqduq create [OPTIONS]
Try 'duqduq create --help' for help.

Error: No such option: --dry_run


## Submit to slurm using `duqduq submit`

Use `duqduq submit` to submit the jobs to slurm. This tool will find all jobs (`.llcmd` files in the subdirectories) and submit them to slurm.

Use the `--array` option to submit the jobs as a slurm array.

In [7]:
!duqtools submit --array --max_jobs 10 --yes --dry_run

15:17:10 [WARNING] Python module 'omas' not found. Submodule 'jams' needs it @jams.py:14
15:17:10 [WARNING] Python module 'netCDF4' not found. Submodule 'transp' needs it @transp.py:25

Operations in the Queue:
- Adding to array : Job('/gss_efgw_work/work/g2ssmee/jetto/runs/duqtools_demo/run_0000')
- Adding to array : Job('/gss_efgw_work/work/g2ssmee/jetto/runs/duqtools_demo/run_0001')
- Adding to array : Job('/gss_efgw_work/work/g2ssmee/jetto/runs/duqtools_demo/run_0002')
- Submit single array job : duqtools_slurm_array.sh
Applying Operations
4

  0%|                                                     | 0/4 [00:00<?, ?it/s]
Submit single array job : duqtools_slurm_array.sh:                                      
Progress: 100%|███████████████████████████████████| 4/4 [00:00<00:00, 50.03it/s]


## `duqduq status`

Query the status using `duqduq status`. This essentially parses all the `jetto.status` files in the run directory.

In [8]:
!duqduq status

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
          216865_0        gw duqtools  g2ssmee  R       0:04      1 r054c05s04
          216865_1        gw duqtools  g2ssmee  R       0:04      1 r054c05s04
          216865_2        gw duqtools  g2ssmee  R       0:04      1 r182c14s04
15:17:16 [WARNING] Python module 'omas' not found. Submodule 'jams' needs it @jams.py:14
15:17:16 [WARNING] Python module 'netCDF4' not found. Submodule 'transp' needs it @transp.py:25
Total number of directories with submit script     : 3
Total number of directories with unsubmitted jobs  : 3
Total number of directories with status script     : 0
Total number of directories with completed status  : 0
Total number of directories with failed status     : 0
Total number of directories with running status    : 0
Total number of directories with unknown status    : 0


## Overview of run directory

Duqtools writes the data in the directory specified by `run_dir` in `duqtools.yaml`. 

- `duqtools.yaml` is a copy of the duqtools config
- `logs` contains the logs from the slurm array
- `runs.yaml` contains the metadata for each of the runs so that they can be reproduced
- `run_000{1..3}` are the jetto run directories. There are 3 directories corresponding to `n_samples: 3` in the duqtools config.

In [9]:
os.chdir(run_dir)
!tree -L 1

.
├── duqtools.yaml
├── logs
├── run_0000
├── run_0001
├── run_0002
└── runs.yaml

4 directories, 2 files


A jetto run directory.

In [10]:
os.chdir(run_dir / 'run_0000')
!tree -L 1

.
├── configs
├── data
├── duqtools.submit.lock
├── envs
├── fort.12
├── fort.13
├── fort.21
├── fort.260 -> jetto.nbip
├── fort.261 -> jetto.nbip1
├── fort.262 -> jetto.nbip2
├── fort.263 -> jetto.nbip3
├── fort.272 -> jetto.tip
├── fort.273 -> jetto.ni1p
├── fort.274 -> jetto.ni2p
├── fort.275 -> jetto.gasp
├── fort.278 -> jetto.eqt
├── fort.280 -> jetto.sadas
├── fort.29 -> jetto.restart
├── fort.300 -> jetto.ext
├── fort.364 -> jetto.beamionsource
├── fort.40 -> jetto.chksum
├── fort.400 -> pion.0
├── fort.401 -> pion.1
├── fort.402 -> pion.2
├── fort.406 -> pion.6
├── fort.407 -> pion.7
├── fort.419 -> pion.19
├── fort.420 -> pion.20
├── fort.45 -> jetto.mhddb
├── fort.451 -> pion.51
├── fort.452 -> pion.52
├── fort.453 -> pion.53
├── fort.454 -> pion.54
├── fort.455 -> pion.55
├── fort.456 -> pion.56
├── fort.470 -> pion.70
├── fort.471 -> pion.71
├── fort.472 -> pion.72
├── fort.48 -> configs/eigen
├── fort.491 -> configs/jht
├── fort.492 -> jetto.jht
├── fort.493 -> configs/jhp

The data are stored in the local imasdb (support for JINTRAC v220922+).

In [11]:
!tree imasdb

imasdb
└── jet
    └── 3
        ├── 0
        │   ├── ids_903500001.characteristics
        │   ├── ids_903500001.datafile
        │   ├── ids_903500001.tree
        │   ├── ids_903500002.characteristics
        │   ├── ids_903500002.datafile
        │   └── ids_903500002.tree
        ├── 1
        ├── 2
        ├── 3
        ├── 4
        ├── 5
        ├── 6
        ├── 7
        ├── 8
        └── 9

12 directories, 6 files


## Data visualization with `duqtools plot`

Duqtools contains a tool to create plots for your data from the command line.

In [12]:
os.chdir(duqtools_dir)

!duqtools plot -i data.csv -v t_i_ave -v t_e -v zeff

You can now view your plot in your browser:

  rho_tor_norm vs. t_i_ave:

    file:////afs/eufus.eu/g2itmuse/user/g2ssmee/duqtools_demo/chart_rho_tor_norm-t_i_ave.html

You can now view your plot in your browser:

  rho_tor_norm vs. t_e:

    file:////afs/eufus.eu/g2itmuse/user/g2ssmee/duqtools_demo/chart_rho_tor_norm-t_e.html

You can now view your plot in your browser:

  rho_tor_norm vs. zeff:

    file:////afs/eufus.eu/g2itmuse/user/g2ssmee/duqtools_demo/chart_rho_tor_norm-zeff.html



## Data exploration with `duqtools dash`

This is a tool for data exploration and visualization.

Use this `data.csv`:  
`/afs/eufus.eu/user/g/g2ssmee/jetto_runs/workspace_demo/data.csv`

In [13]:
!duqtools dash


  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://130.186.25.53:8501

^C
  Stopping...


## Canonical UQ with `duqtools setup`

Duqtools setup is a tool that can help with canonical UQ.

It takes a templated called `duqtools.template.yaml` which contains the canonical settings. `duqtools setup` uses this file to generate a `duqtools.yaml` file, which is compatible with `duqtools create` (see steps above).

You can specify the input imas data on the command line. Duqtools will fill some machine specific from the IDS directly. The rules are specified in `variables_ids2jetto.yaml`. See the [documentation](https://duqtools.readthedocs.io/en/latest/variables/#ids2jetto-variables) for more information about variable specification.

In [14]:
import os
os.chdir('/afs/eufus.eu/user/g/g2ssmee/duqtools_template')

Below is what the template looks like.

The template uses [jinja2](https://jinja.palletsprojects.com/en/latest/) as a templating language.

Accolates (`{..}`) denote placeholders that will be replaced by the templating `jinja2`. Three objects are currently available (`run` to denote the run name, `handle` to reference the imas data, and `variables` to reference variables from the IDS data. See the [documentation](https://duqtools.readthedocs.io/en/latest/large_scale_validation/#placeholder-variables) for more information about these placeholders. 

The template supports functions to modify the placeholders or do some simple logic. The example below does some rounding of the variables using the `round()` function. The end time is calculated by taking the start time and adding 0.01 s.

In [15]:
%cat duqtools.template.yaml

tag: {{ run.name }}
create:
  runs_dir: /afs/eufus.eu/user/g/g2ssmee/jetto_runs/{{ run.name }}
  template: /afs/eufus.eu/user/g/g2ssmee/jetto_runs/interpretive_esco02
  template_data:
    user: {{ handle.user }}
    db: {{ handle.db }}
    shot: {{ handle.shot }}
    run: {{ handle.run }}
  sampler:
    method: latin-hypercube
    n_samples: 25
  dimensions:
    - variable: zeff
      operator: multiply
      values: [0.8, 0.9, 1.0, 1.1, 1.2]
    - variable: t_e
      operator: multiply
      values: [0.8, 0.9, 1.0, 1.1, 1.2]
    - variable: major_radius
      operator: copyto
      {# Convert units from IDS (m) to Jetto.jset (cm) -#}
      values: [ {{ (variables.major_radius * 100) | round(4) }} ]
    - variable: b_field
      operator: copyto
      values: [ {{ variables.b_field | round(4) }} ]
    - variable: t_start
      operator: copyto
      values: [ {{ variables.t_start | round(4) }} ]
    - variable: t_end
      operator: copyto
      values: [ {{ (variables.t_start + 0.01) 

Using `duqtools setup` replaces the `handle` attributes with the imas location given on the CLI, and the `run.name` with the name of the run.

In [16]:
!duqtools setup --handle g2aho/jet/90350/2 --run_name my_run --yes


Operations in the Queue:
- Setup run : my_run
Applying Operations
1

  0%|                                                     | 0/1 [00:00<?, ?it/s]
Setup run : my_run: 
Progress: 100%|██████████████████████████████████| 1/1 [00:00<00:00, 415.03it/s]


Below is what the resulting `duqtools.yaml` looks like. It can be modified or directly used as input for `duqtools create`.

In [17]:
%cat my_run/duqtools.yaml

tag: my_run
create:
  runs_dir: /afs/eufus.eu/user/g/g2ssmee/jetto_runs/my_run
  template: /afs/eufus.eu/user/g/g2ssmee/jetto_runs/interpretive_esco02
  template_data:
    user: g2aho
    db: jet
    shot: 90350
    run: 2
  sampler:
    method: latin-hypercube
    n_samples: 25
  dimensions:
    - variable: zeff
      operator: multiply
      values: [0.8, 0.9, 1.0, 1.1, 1.2]
    - variable: t_e
      operator: multiply
      values: [0.8, 0.9, 1.0, 1.1, 1.2]
    - variable: major_radius
      operator: copyto
      values: [ 296.0 ]
    - variable: b_field
      operator: copyto
      values: [ 2.1155 ]
    - variable: t_start
      operator: copyto
      values: [ 45.75 ]
    - variable: t_end
      operator: copyto
      values: [ 45.76 ]
system: jetto-v220922